# Sequential Resource Holding and Blocking in SimPy

In this notebook we will learn how to code simpy logic to mimic a process holding a resource while queuing for another resource.  This simulates a blocking scenario in a process.

We will work with a hypothetical stroke pathway where patients undergo acute treatment followed by transfer at a different hospital to undergo rehabilitation.  A patient must remain in an acute stroke bed until a rehabilitation bed is free.

> In this example we will not concern ourselves with a warm-up period or initial conditions.

![model image](img/bed_blocking_image.png "Bed blocking example")


## 1. Imports 

In [1]:
import numpy as np
import itertools
import simpy

In [2]:
# to reduce code these classes can be found in distribution.py
from distributions import Exponential, Lognormal

## 2. Constants

In [3]:
# default mean inter-arrival times(exp)
# time is in days
IAT_STROKES = 1.5

# resources
N_ACUTE_BEDS = 9
N_REHAB_BEDS = 15

# Acute LoS (Lognormal)
ACUTE_LOS_MEAN = 7.0
ACUTE_LOC_STD = 1.0

# Rehab LoS (Lognormal)
REHAB_LOS_MEAN = 30.0
REHAB_LOC_STD = 5.0

# sampling settings
N_STREAMS = 3
DEFAULT_RND_SET = 0

# Boolean switch to simulation results as the model runs
TRACE = False

# run variables (units = days)
RUN_LENGTH = 100

## 2. Helper classes and functions

In [4]:
def trace(msg):
    """
    Turing printing of events on and off.

    Params:
    -------
    msg: str
        string to print to screen.
    """
    if TRACE:
        print(msg)

## 3. Experiment class

In [5]:
class Experiment:
    """
    Encapsulates the concept of an experiment 🧪 for the stroke pathway
    bed blocking simulator. Manages parameters, PRNG streams and results.
    """
    def __init__(
        self,
        random_number_set=DEFAULT_RND_SET,
        n_streams=N_STREAMS,
        iat_strokes=IAT_STROKES,
        acute_los_mean=ACUTE_LOS_MEAN,
        acute_los_std=ACUTE_LOC_STD,
        rehab_los_mean=REHAB_LOS_MEAN,
        rehab_los_std=REHAB_LOC_STD,
        n_acute_beds=N_ACUTE_BEDS,
        n_rehab_beds=N_REHAB_BEDS
    ):
        """
        The init method sets up our defaults.
        """
        # sampling
        self.random_number_set = random_number_set
        self.n_streams = n_streams

        # store parameters for the run of the model
        self.iat_strokes = iat_strokes
        self.acute_los_mean = acute_los_mean
        self.acute_los_std = acute_los_std
        self.rehab_los_mean = rehab_los_mean
        self.rehab_los_std = rehab_los_std

        #  place holder for resources
        self.acute_ward = None
        self.rehab_unit = None
        self.n_acute_beds = n_acute_beds
        self.n_rehab_beds = n_rehab_beds
        
        # initialise results to zero
        self.init_results_variables()

        # initialise sampling objects
        self.init_sampling()

    def set_random_no_set(self, random_number_set):
        """
        Controls the random sampling
        Parameters:
        ----------
        random_number_set: int
            Used to control the set of pseudo random numbers used by
            the distributions in the simulation.
        """
        self.random_number_set = random_number_set
        self.init_sampling()

    def init_sampling(self):
        """
        Create the distributions used by the model and initialise
        the random seeds of each.
        """
        # produce n non-overlapping streams
        seed_sequence = np.random.SeedSequence(self.random_number_set)
        self.seeds = seed_sequence.spawn(self.n_streams)

        # create distributions

        # inter-arrival time distributions
        self.arrival_strokes = Exponential(
            self.iat_strokes, random_seed=self.seeds[0]
        )

        self.acute_los = Lognormal(
            self.acute_los_mean, self.acute_los_std, random_seed=self.seeds[1]
        )

        self.rehab_los = Lognormal(
            self.rehab_los_mean, self.rehab_los_std, random_seed=self.seeds[2]
        )

    def init_results_variables(self):
        """
        Initialise all of the experiment variables used in results
        collection.  This method is called at the start of each run
        of the model
        """
        # variable used to store results of experiment
        self.results = {}
        self.results["n_arrivals"] = 0
        self.results["waiting_acute"] = []
        self.results["bed_blocking_times"] = []

## 4. Pathway process logic

The key things to recognise are 

* We request the bed from the acute stroke unit as usual using a `with` context manager
* We request the rehab bed within the acute bed `with` context manager.  This means we do not release the acute bed while the patient waits for rehab.
* As we do not use a `with` context manager for rehab there is no teardown and we need to manually release the rehab bed.

In [6]:
def patient_pathway(patient_id, env, args):
    """Process a patient through the acute ward and rehab unit.
    Simpy generator function.
    
    Parameters:
    -----------
    patient_id: int
        A unique id representing the patient in the process

    env: simpy.Environment
        The simulation environment

    args: Experiment
        Container class for the simulation parameters/results.
    """
    arrival_time = env.now

    with args.acute_ward.request() as acute_bed_request:
        yield acute_bed_request
        
        acute_admit_time = env.now
        wait_for_acute = acute_admit_time - arrival_time
        args.results['waiting_acute'].append(wait_for_acute)
        
        trace(f"{env.now:.2f}: Patient {patient_id} admitted to acute ward." \
              + f"(waited {wait_for_acute:.2f} days)")
        
        # Simulate acute care treatment
        acute_care_duration = args.acute_los.sample()
        yield env.timeout(acute_care_duration)
        
        # Patient is now medically ready for rehabilitation
        medically_ready_time = env.now
        trace(f"{env.now:.2f}: Patient {patient_id} medically ready for rehab")
        
        # Request a rehab bed but don't release the acute bed immediately
        # Note we are still within the "with" context manager for the acute bed
        # This is where bed blocking occurs. We wait here until the rehab bed
        # is available. Make sure the indentation is correct or you will release
        rehab_bed = args.rehab_unit.request()
        yield rehab_bed
        
        # Now we have a rehab bed, we can transfer the patient
        transfer_time = env.now
        bed_blocking_duration = transfer_time - medically_ready_time
        args.results['bed_blocking_times'].append(bed_blocking_duration)
        
        trace(f"{env.now:.2f}: Patient {patient_id} transferred to rehab. "\
            + f"(blocked acute bed for {bed_blocking_duration:.2f} days)")
        
    # Acute bed is now released
    # Note the indentation!  We are now outside of the with context manager.
    # This automatically releases the simpy resource.
    
    # Simulate rehabilitation stay
    rehab_duration = args.rehab_los.sample()
    yield env.timeout(rehab_duration)
    
    # Patient completes rehabilitation and is discharged
    discharge_time = env.now

    # Note: we need to explicitly call release on the rehab resource.
    args.rehab_unit.release(rehab_bed)

    trace(f"{env.now:.2f}: Patient {patient_id} discharged from Rehab.")

## 4. Arrivals generator

This is a standard arrivals generator. We create stroke arrivals according to their distribution.

In [7]:
def stroke_arrivals_generator(env, args):
    """
    Arrival process for strokes.

    Parameters:
    ------
    env: simpy.Environment
        The simpy environment for the simulation

    args: Experiment
        The settings and input parameters for the simulation.
    """
    # use itertools as it provides an infinite loop
    # with a counter variable that we can use for unique Ids
    for patient_id in itertools.count(start=1):

        # the sample distribution is defined by the experiment.
        inter_arrival_time = args.arrival_strokes.sample()
        yield env.timeout(inter_arrival_time)

        args.results["n_arrivals"] = patient_id
        
        trace(f"{env.now:.2f}: Stroke arrival.")

        # patient enters pathway
        env.process(patient_pathway(patient_id, env, args))

## 5. Single run function

In [8]:
def single_run(
    experiment, 
    rep=0,
    run_length=RUN_LENGTH
):
    """
    Perform a single run of the model and return the results

    Parameters:
    -----------

    experiment: Experiment
        The experiment/paramaters to use with model

    rep: int
        The replication number.

    rc_period: float, optional (default=RUN_LENGTH)
        The run length of the model
    """

    # reset all results variables to zero and empty
    experiment.init_results_variables()

    # set random number set to the replication no.
    # this controls sampling for the run.
    experiment.set_random_no_set(rep)

    # environment is (re)created inside single run
    env = simpy.Environment()

    # simpy resources
    experiment.acute_ward = simpy.Resource(env, experiment.n_acute_beds)
    experiment.rehab_unit = simpy.Resource(env, experiment.n_rehab_beds)

    # we pass all arrival generators to simpy 
    env.process(stroke_arrivals_generator(env, experiment))

    # run model
    env.run(until=run_length)

    # quick stats
    results = {}
    results['mean_acute_wait'] = np.array(
        experiment.results["waiting_acute"]
    ).mean()
    results['mean_bed_blocking'] = np.array(
        experiment.results["bed_blocking_times"]
    ).mean()

    # return single run results
    return results

In [9]:
TRACE = True
experiment = Experiment()
results = single_run(experiment)
results

4.94: Stroke arrival.
4.94: Patient 1 admitted to acute ward.(waited 0.00 days)
6.08: Stroke arrival.
6.08: Patient 2 admitted to acute ward.(waited 0.00 days)
7.96: Stroke arrival.
7.96: Patient 3 admitted to acute ward.(waited 0.00 days)
8.30: Stroke arrival.
8.30: Patient 4 admitted to acute ward.(waited 0.00 days)
8.63: Stroke arrival.
8.63: Patient 5 admitted to acute ward.(waited 0.00 days)
11.33: Stroke arrival.
11.33: Patient 6 admitted to acute ward.(waited 0.00 days)
11.37: Patient 2 medically ready for rehab
11.37: Patient 2 transferred to rehab. (blocked acute bed for 0.00 days)
11.43: Stroke arrival.
11.43: Patient 7 admitted to acute ward.(waited 0.00 days)
12.18: Patient 3 medically ready for rehab
12.18: Patient 3 transferred to rehab. (blocked acute bed for 0.00 days)
12.71: Patient 1 medically ready for rehab
12.71: Patient 1 transferred to rehab. (blocked acute bed for 0.00 days)
12.78: Stroke arrival.
12.78: Patient 8 admitted to acute ward.(waited 0.00 days)
13.23:

{'mean_acute_wait': 1.041965345852034, 'mean_bed_blocking': 4.3271192681703115}

## Quick tests

### Remove rehab bed blocking

In [10]:
M = 1_000_000
TRACE = False
experiment = Experiment(n_rehab_beds=M)
results = single_run(experiment)
results

{'mean_acute_wait': 0.0, 'mean_bed_blocking': 0.0}

### Remove all waiting time for acute unit

In [11]:
M = 1_000_000
TRACE = False
experiment = Experiment(n_acute_beds=M)
results = single_run(experiment)
results

{'mean_acute_wait': 0.0, 'mean_bed_blocking': 4.80862454209246}